##### Import libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader

/home/felix.blanco/.conda/envs/BT/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### Import own libraries

In [2]:
from dataloader.iris import IrisDataset
from model.iris_cnn import IrisCNN

##### Load dataset

In [3]:
iris_df = pd.read_csv('../data/iris.data', header = None)

# Divide the dataset into 'features' and 'labels' arrays
features = np.array(iris_df.iloc[:, :-1])
labels = np.array(iris_df.iloc[:, -1])

# Encoder iris labels
label_names = np.unique(labels)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(label_names)
label_dict = {}
for label, encoded_label in zip(label_names, encoded_labels):
   label_dict[label] = encoded_label

# Map Encoded labels to DF
labels = label_encoder.transform(labels)
print('Features shape:', features.shape, 'Labels shape:', labels.shape)

Features shape: (150, 4) Labels shape: (150,)


In [4]:
# Split the dataset into train and test sets
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size = 0.2, random_state = 42
)

##### Dataloader

In [5]:
batch_size = 16
train_dataset = IrisDataset(train_features, train_labels)
test_dataset = IrisDataset(test_features, test_labels)
# Create a DataLoader
train_dataloader = DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True, num_workers = 4
)
test_dataloader = DataLoader(
    test_dataset, batch_size = batch_size, shuffle = True, num_workers = 4
)

In [6]:
model = IrisCNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

##### Trainning

In [7]:
num_epochs = 100
for epoch in range(num_epochs):
    for features, labels in train_dataloader:
        optimizer.zero_grad()
        # Forward pass
        outputs = model(features)
        loss = loss_fn(outputs, labels)
        # Backward and optimize
        loss.backward()
        optimizer.step()
    
    # Print the training loss after each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/100], Loss: 0.9585
Epoch [2/100], Loss: 0.9319
Epoch [3/100], Loss: 0.8278
Epoch [4/100], Loss: 0.6928
Epoch [5/100], Loss: 0.4885
Epoch [6/100], Loss: 0.5858
Epoch [7/100], Loss: 0.4286
Epoch [8/100], Loss: 0.4251
Epoch [9/100], Loss: 0.1934
Epoch [10/100], Loss: 0.2730
Epoch [11/100], Loss: 0.4672
Epoch [12/100], Loss: 0.1625
Epoch [13/100], Loss: 0.2554
Epoch [14/100], Loss: 0.0907
Epoch [15/100], Loss: 0.0456
Epoch [16/100], Loss: 0.1212
Epoch [17/100], Loss: 0.0430
Epoch [18/100], Loss: 0.0600
Epoch [19/100], Loss: 0.0557
Epoch [20/100], Loss: 0.1507
Epoch [21/100], Loss: 0.0601
Epoch [22/100], Loss: 0.0356
Epoch [23/100], Loss: 0.0823
Epoch [24/100], Loss: 0.0297
Epoch [25/100], Loss: 0.0248
Epoch [26/100], Loss: 0.0447
Epoch [27/100], Loss: 0.0079
Epoch [28/100], Loss: 0.1355
Epoch [29/100], Loss: 0.1144
Epoch [30/100], Loss: 0.0991
Epoch [31/100], Loss: 0.0517
Epoch [32/100], Loss: 0.0301
Epoch [33/100], Loss: 0.0374
Epoch [34/100], Loss: 0.0120
Epoch [35/100], Loss: 0

##### Testing

In [8]:
with torch.no_grad():
    correct = 0
    total = 0
    for features, labels in test_dataloader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 96.66666666666667%
